<a href="https://colab.research.google.com/github/DevCielo/neural-networks-from-scratch/blob/main/Neural_Networks_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bulding CNN's with tensorflow and from scratch**

# LeNet-5 with tensorflow


In [8]:
import tensorflow as tf

def LeNet5():
  model = tf.keras.Sequential([
      # Convolution Layer
      tf.keras.layers.Conv2D(6, (5, 5), activation='tanh', strides=(1, 1), padding='valid', name='conv0'),
      # Average Pooling Layer
      tf.keras.layers.AveragePooling2D((2,2), strides=(2,2), name='avg_pool0'),
      # Convolution Layer
      tf.keras.layers.Conv2D(16, (5, 5), strides=(1,1), padding='valid', activation='tanh', name='conv1'),
      # Average Pooling Layer
      tf.keras.layers.AveragePooling2D((2,2), strides=(2,2), name='avg_pool1'),
      # Convolution Layer
      tf.keras.layers.Conv2D(120, (5, 5), strides=(1, 1), padding='valid', activation='tanh', name='conv2'),
      # Flatten the output
      tf.keras.layers.Flatten(),
      # Fully Connected Layer
      tf.keras.layers.Dense(84, activation='tanh', name='fc1'),
      tf.keras.layers.Dense(10, activation='softmax', name='output'),
  ])

  return model

# LeNet-5 from scratch

In [13]:
import numpy as np

def conv2d(input, filters, bias, stride=1, padding=0):
  # decompose the filters shape into the filter size (f) and the number of filters (n_f)
  (n_f, f, f) = filters.shape
  # decomposes the input shape into the input dimensions (in_dim) and the number of channels (n_c)
  (in_dim, in_dim, n_c) = input.shape

  # calculates the output dimensions using formula ((n+2p-f)/s + 1)
  # int to floor function
  out_dim = int((in_dim + 2*padding-f)/stride) + 1

  # sets the new output shape to be out_dim*out_dim*n_f
  output = np.zeros((out_dim, out_dim, n_f))

  # adds padding to the height and width but none to the number of channels
  input_padded = np.pad(input, ((padding, padding), (padding, padding), (0, 0)), mode='constant', constant_values = (0,0))

  # for every filter, performs the convolution by calculated the height/width start and ends and suming over their values to produce an output
  for i in range(n_f):
    for h in range(out_dim):
      for w in range(out_dim):
        h_start = h*stride
        h_end = h_start + f
        w_start = w*stride
        w_end = w_start + f
        output[h, w, i] = np.sum(input_padded[h_start:h_end, w_start:w_end, :] * filters[i]) + bias[i]

  return output


(32, 32, 6)


In [14]:
# Example usage of conv2d function (Equal to first step in LeNet-5)
input = np.random.randn(32, 32, 1)  # Example input
filters = np.random.randn(6, 5, 5)  # Example filters
bias = np.random.randn(6)           # Example bias
output = conv2d(input, filters, bias, stride=1, padding=0)
print(output.shape)  # Expected Output: (28, 28, 6)

(28, 28, 6)
